In [1]:
import requests
import pandas

In [2]:
ENDPOINT: str = "https://search.dip.bundestag.de/api/v1/person?format=json&apikey=I9FKdCn.hbfefNWCY336dL6x62vfwNKpoN2RZ1gp21"

In [3]:
def get_data(cursor: str | None = None):
    if cursor:
        return requests.get(f"{ENDPOINT}&cursor={cursor}").json()
    
    return requests.get(ENDPOINT).json()

In [ ]:
data = []

response = get_data()
cursor = response["cursor"]

for person in response["documents"]:
    data.append(person)


while cursor:
    response = get_data(cursor)

    if response["cursor"] != cursor:
        cursor = response["cursor"]
    
    else:
        cursor = None

    for person in response["documents"]:
        data.append(person)


data

In [ ]:
pandas.DataFrame(data)

In [ ]:
(
    pandas.DataFrame(data)
    [["nachname", "vorname", "titel", "wahlperiode"]]
    .pipe(lambda _df: _df[_df["wahlperiode"] == 20])
    .pipe(lambda _df: _df[_df["titel"].str.contains("MdB,")])
    .pipe(lambda _df: _df.assign(fraktion=_df["titel"].str.split().str[-1]))
    .drop(columns=["titel", "wahlperiode"])
)